In [1]:
import streamlit as st
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import time
from datetime import datetime, timedelta
from EV_class import EV_BATTERY

In [7]:
print(datetime(2024, 6, 19, 18, 0))
a = datetime(2024, 6, 19, 18, 0)
b = datetime(2024, 6, 20, 7, 30)


tuple_time = (a, b)
c = b - a 
d = tuple_time[1] - tuple_time[0]

2024-06-19 18:00:00


In [3]:
print(int(c.total_seconds()/(60*10)))
print(type(c))

3
<class 'datetime.timedelta'>


In [8]:
Batt = EV_BATTERY(P_charg=46, capacity=62, P_dischar=7, eta = 0.8, n_EV=3)

Batt.read_spot_price('Price_data/bourseEpex_06_06_2024.csv', resample=True)
Batt.reformate_charing_time(tuple_time)
matrix = Batt.generate_contiguous_matrix(SOC_random=False,
                                         SOC_ini=0.5*np.ones((3,)),
                                         SOC_final=0.95*np.ones((3,)))
charge, discharge, SOC = Batt.battery_dispatch('VPP')
total_price= Batt.obj_value

df = Batt.generate_result_df()
print(df)

Not random
Not random
Not random
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[rosetta2])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1299 rows, 1728 columns and 3072 nonzeros
Model fingerprint: 0xd6909477
Model has 864 quadratic constraints
Variable types: 1296 continuous, 432 integer (432 binary)
Coefficient statistics:
  Matrix range     [2e-01, 6e+01]
  QMatrix range    [4e+02, 4e+02]
  QLMatrix range   [1e+00, 4e+02]
  Objective range  [2e-03, 1e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e-01, 5e+01]
  QRHS range       [4e+02, 4e+02]
Presolve removed 997 rows and 1422 columns
Presolve time: 0.02s
Presolved: 446 rows, 354 columns, 1081 nonzeros
Variable types: 273 continuous, 81 integer (81 binary)
Found heuristic solution: objective 0.2245270

Root relaxation: objective 1.460949e+00, 78 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective 

In [11]:
fig, ax = plt.subplots(figsize = (5,3), dpi = 300)

#ls_time = df['Time']

ls_time = [i for i in range(len(Batt.price_ls))]
#ax2 = ax.twinx()


#ax2.plot(ls_time, SOC[:,0], label = 'VPP', color ='red')
ax.plot(ls_time, Batt.price_ls, label = 'Price', linestyle='--')
ax.legend()
ax.set_ylabel('Price [$/MWh]')
#ax2.set_ylabel('State of charge')
#ax2.legend(loc = 'lower right')



Text(0, 0.5, 'Price [$/MWh]')

In [12]:
print(SOC)

[[0.5        0.5        0.5       ]
 [0.4811828  0.4811828  0.4811828 ]
 [0.46236559 0.46236559 0.46236559]
 [0.44354839 0.44354839 0.44354839]
 [0.42473118 0.42473118 0.42473118]
 [0.40591398 0.40591398 0.40591398]
 [0.38709677 0.38709677 0.38709677]
 [0.36827957 0.36827957 0.36827957]
 [0.34946237 0.34946237 0.34946237]
 [0.33064516 0.33064516 0.33064516]
 [0.31182796 0.31182796 0.31182796]
 [0.29301075 0.29301075 0.29301075]
 [0.27419355 0.27419355 0.27419355]
 [0.25537634 0.25537634 0.25537634]
 [0.23655914 0.23655914 0.23655914]
 [0.21774194 0.21774194 0.21774194]
 [0.19892473 0.19892473 0.19892473]
 [0.18010753 0.18010753 0.18010753]
 [0.16129032 0.16129032 0.16129032]
 [0.16129032 0.16129032 0.16129032]
 [0.16129032 0.16129032 0.16129032]
 [0.16129032 0.16129032 0.16129032]
 [0.16129032 0.16129032 0.16129032]
 [0.16129032 0.16129032 0.16129032]
 [0.16129032 0.16129032 0.16129032]
 [0.28494624 0.16129032 0.28494624]
 [0.40860215 0.19354839 0.40860215]
 [0.40860215 0.3172043  0.40

In [1]:
import streamlit as st
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import time
from datetime import datetime, timedelta
from EV_class import EV_BATTERY
import plotly.express as px

# Set the page layout
st.set_page_config(layout="wide")

# Sidebar for input sliders
st.sidebar.title("Input Sliders")

slider_values = []
for i in range(1, 6):
    value = st.sidebar.slider(f'Slider {i}', 0, 100, 50)
    slider_values.append(value)


# State of charge data
SOC_ini = st.sidebar.slider(f'Initial State of Charge', 0, 100, 50)
SOC_final = st.sidebar.slider(f'Desired State of Charge', 0, 100, 100)
    

start_time = datetime.now().replace(hour=18, minute=0, second=0, microsecond=0)
end_time = start_time + timedelta(days=1, hours=-1)

# Create the slider charging schedule timeline
charging_schedule = st.sidebar.slider(
    "Schedule your appointment:",
    min_value=start_time,
    max_value=end_time,
    value=(start_time + timedelta(hours=0, minutes=0), start_time + timedelta(hours=5, minutes=45)),
    step=timedelta(minutes=10),
    format="MM/DD HH:mm"
)
st.write("You're scheduled for:", charging_schedule)
print(charging_schedule)

#=====================================================================================
#============= Run simulation ========================================================
#=====================================================================================

Batt = EV_BATTERY(P_charg=46, capacity=62, P_dischar=7, eta = 0.8, n_EV=3)

Batt.read_spot_price('Price_data/bourseEpex_06_06_2024.csv', resample=True)
Batt.reformate_charing_time(charging_schedule)

print(Batt.start_pos)
print(Batt.len_ones)
Batt.generate_contiguous_matrix(SOC_random=False,
                                         SOC_ini= SOC_ini*np.ones((3,)),
                                         SOC_final= SOC_final*np.ones((3,)))
charge, discharge, SOC = Batt.battery_dispatch('VPP')
total_price= Batt.obj_value

df = Batt.generate_result_df()

# Main dashboard layout
col1, col2 = st.columns(2)

# Top-left: Text and number
col1.subheader("Charging operation for Nissan LEAF")
col1.metric(label="Charging cost", value=SOC_ini)
col1.metric(label="Charging cost", value=SOC_ini)

# Top-right: Bar plot
col2.subheader("Bar Plot")
fig, ax = plt.subplots()
ax.bar(range(1, 6), slider_values)
ax.set_xlabel("Slider")
ax.set_ylabel("Value")
col2.pyplot(fig)

# Bottom: Line plot
st.subheader("Line Plot")


fig = px.scatter(df, x="Time", y="Price [$/MWh]")

event = st.plotly_chart(fig, on_select="rerun")


2024-06-19 17:17:27.899 
  command:

    streamlit run /Users/adrienguenard/opt/anaconda3/envs/stochastic/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]


(datetime.datetime(2024, 6, 19, 18, 0), datetime.datetime(2024, 6, 19, 19, 45))
0
10
Not random
Not random
Not random
Set parameter Username
Academic license - for non-commercial use only - expires 2024-09-07
Gurobi Optimizer version 10.0.0 build v10.0.0rc2 (mac64[rosetta2])

CPU model: Apple M2
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1299 rows, 1728 columns and 2646 nonzeros
Model fingerprint: 0x195298c0
Model has 864 quadratic constraints
Variable types: 1296 continuous, 432 integer (432 binary)
Coefficient statistics:
  Matrix range     [2e-01, 6e+01]
  QMatrix range    [4e+02, 4e+02]
  QLMatrix range   [1e+00, 4e+02]
  Objective range  [2e-03, 1e-02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [5e+01, 1e+02]
  QRHS range       [4e+02, 4e+02]
Presolve removed 804 rows and 1206 columns
Presolve time: 0.00s

Explored 0 nodes (0 simplex iterations) in 0.00 seconds (0.00 work units)
Thread count was 1 (of 8 available pr

AttributeError: Unable to retrieve attribute 'X'